# Day 24

link: https://adventofcode.com/2022/day/24

ข้อนี้ก็เป็นการหา shortest path อีกแล้ว เพียงแต่ว่า map ของเรามันจะเปลี่ยนไปในแต่ละ time step

เราคิดว่าวิธีที่ implement ง่ายที่สุด ก็คือการคิด map ล่วงหน้าไว้ทุก time step เลย
(หรือจะจินตนาการว่ามันเป็น 3-d spacetime map ก็ได้)

ใน map ขนาด r x c สังเกตว่า blizzard ในแนวนอนจะวนซ้ำทุก c steps
และในแนวตั้งจะวนซ้ำทุก r steps ดังนั้นทั้ง map จะวนกลับเข้า state เดิมใน L = lcm(r, c) steps
เราก็เตรียม map ไว้ล่วงหน้าทั้ง L steps

In [1]:
input = IO.foreach('data/24.txt').to_a.map(&:strip)

h = input.size - 2
w = input[0].size - 2
l = h.lcm(w)

a = Array.new(l){ Array.new(h) {Array.new(w, true)}}

h.times{|r|
  w.times{|c|
    d = input[r+1][c+1]
    next if d == '.'

    dr, dc = case d
    when '>' then [0, 1]
    when '<' then [0, -1]
    when 'v' then [1, 0]
    when '^' then [-1, 0]
    end

    l.times{|t|
      a[t][(l+r+dr*t) % h][(l+c+dc*t) % w] = false
    }
  }
}
nil

## Part 1

เราจะใช้วิธีเก็บ reachable states ในแต่ละ time step ว่าเราสามารถเดินไปถึงพิกัดไหนได้บ้าง
พอขยับไป time step ถัดไป เราก็เปลี่ยน map ด้วย แล้วดูว่าจากแต่ละ state จะเคลื่อนไปตรงไหนได้
ซึ่งอาจจะเดินขึ้น ลง ซ้าย ขวา หรืออยู่ที่เดิมก็ได้

นอกจากนี้เรายังสามารถหยุดรออยู่ที่จุดเริ่มต้นนานเท่าไหร่ก็ได้ ดังนั้นทุกๆ time step เราต้องเช็คว่าจุดเริ่มต้น (0, 0) นั้นว่างหรือเปล่า
ถ้าว่างมันก็จะกลายเป็น reachable state ใน time step นั้นๆ ด้วย

In [2]:
require 'set'

def can_move(mp, r, c)
  r >= 0 && r < mp.size && c >= 0 && c < mp[0].size && mp[r][c]
end

def min_steps(a, initial_slice, start, target)

  positions = Set.new
  turns = 0
  slice = initial_slice
  l = a.size
  until positions.include?(target)
    mp = a[slice]
    n_positions = Set.new
    n_positions << start if mp[start[0]][start[1]]
    positions.each{|r, c|
      n_positions << [r, c] if can_move(mp, r, c)
      n_positions << [r+1, c] if can_move(mp, r+1, c)
      n_positions << [r-1, c] if can_move(mp, r-1, c)
      n_positions << [r, c+1] if can_move(mp, r, c+1)
      n_positions << [r, c-1] if can_move(mp, r, c-1)
    }
    turns += 1
    slice = (slice + 1) % l 
    positions = n_positions
  end
  [turns, slice]
end

turns1, slice1 = min_steps(a, 0, [0, 0], [h-1, w-1])
puts turns1

238


## Part 2

ใน part หลัง เราก็ใช้ method เดิม แต่เปลี่ยนจุดเริ่มต้นและเป้าหมาย และ time step เริ่มต้น
เพิ่อเดินกลับไปที่ (0, 0) และกลับมาอีกครั้ง

In [3]:
turns2, slice2 = min_steps(a, slice1, [h-1, w-1], [0, 0])
turns3, slice3 = min_steps(a, slice2, [0, 0], [h-1, w-1])
puts turns1 + turns2 + turns3

751
